In [1]:

import pickle

model_name = 'gpt3.5-zero-shot-paul-pool-temp0-'

data_path = 'data/data.txt'
path_output_pkl = 'LLM4Eval/outputs/'+model_name+'.pkl'
output_text_path = 'LLM4Eval/outputs/'+model_name+'.txt'


FROM_CHECKPOINT= False
model_id = "gpt-3.5-turbo-0125"
API_key = ""


In [2]:
# Create the prompt per each turn

def create_data_zero_shot(): 
    prompt = """You are a search quality rater evaluating the relevance of web pages. 
    Given the persona of the user, user query, and a web page, you must provide a score on an integer scale of 0 to 4 to indicate to what extent the given document meets the information needs of the user. The scores have the following meanings:

    0: fails to meet 
    1: slightly meets
    2: moderately meets
    3: highly meets
    4: fully meets

    User persona: {ptkb}
    Query: {utterance}
    Passage : {passage}
    Score:
    """
    



    with open(data_path, 'r') as f:
        lines = f.readlines()

    flattened_turn_pool_score = []

    for line in lines:
        turn_id, rewritten_utterance, response_txt,  passage_id, passage_text, score,  ptkb, _ =line.split('\t')
        x = {'turn_id': turn_id,
        'doc_id': passage_id,
        'passage': passage_text,
        'score': score}
        x['prompt'] = prompt.format(ptkb=ptkb, utterance=rewritten_utterance, passage=passage_text)
        flattened_turn_pool_score.append(x)
    
    return flattened_turn_pool_score


flattened_turn_pool_score =  create_data_zero_shot()

if FROM_CHECKPOINT:
    print('Loading from checkpoint')
    with open(path_output_pkl, 'rb') as f:
        flattened_turn_pool_score = pickle.load(f)



In [3]:
from openai import OpenAI

def chatgpt_conversation(conversation_Log):
  response = client.chat.completions.create(
      model = model_id,
      messages= conversation_Log,
      temperature= 0,
      top_p= 1,
      
  )
  
  response= response.choices[0].message.content

  return response

def run_one_sample(prompt):

    conversations = []
    conversations.append({'role': 'user', 'content': prompt})
    response = chatgpt_conversation(conversations)
    score = response.strip()

    # print()
    # print('{0}: {1}\n'.format(conversations[-1]['role'].strip(), conversations[-1]['content'].strip()))

    return score


client = OpenAI(
  api_key=API_key,  # this is also the default, it can be omitted
)


In [ ]:
counter = 0
index = 0

for entry in flattened_turn_pool_score:
    index += 1
    print(index)

    if 'gpt3.5-score' in entry:
        print(entry['gpt3.5-score'])
        print('&&&&&&&&&&&&&&')
        continue
    
    else:
        try:
            pred_score = run_one_sample(entry['prompt'])
            pred_score = pred_score.lower().replace('score:', '').strip()
            entry['gpt3.5-score'] = pred_score
            print(entry['gpt3.5-score'])
            print('****************')
            counter+=1
        except:
            print('unsuccessful')
            entry['gpt3.5-score'] = 'NONE'

    if index%100==99:
        print('100 done')
        with open(path_output_pkl, 'wb') as f:
            pickle.dump(flattened_turn_pool_score, f)

with open(path_output_pkl, 'wb') as f:
    pickle.dump(flattened_turn_pool_score, f)            

In [5]:
with open(path_output_pkl, 'wb') as f:
    pickle.dump(flattened_turn_pool_score, f)

In [ ]:
import re

X = 0
lines = []

for entry in flattened_turn_pool_score:
    if ('gpt3.5-score' in entry):
        numbers = re.findall(r'\d+', entry['gpt3.5-score'].strip())
        if len(numbers)==1:
            score = numbers[0]
        else:
            if ('fails to meet' in entry['gpt3.5-score']):
                score = '0'
            elif ('slightly meets' in entry['gpt3.5-score']):
                score = '1'
            elif ('moderately meets' in entry['gpt3.5-score']):
                score = '2'
            elif ('highly meets' in entry['gpt3.5-score']):
                score = '3'
            elif ('fully meets' in entry['gpt3.5-score']):
                score = '4'
            else:
                tmp = entry['gpt3.5-score'].strip().split('\n')[0]
                numbers = re.findall(r'\d+',  tmp.strip()) 
                if len(numbers)==1:
                    score = numbers[0]
                else:                      
                    score = entry['gpt3.5-score'].strip()
                    print(score)
                    X +=1

        line = entry['turn_id'] + '\t0\t'+ entry['doc_id']+ '\t'+ str(score) + '\n'
        lines.append(line)

print(X)

with open(output_text_path, 'w') as f:
    f.writelines(lines)   
    

In [ ]:
print(output_text_path)